In [ ]:
import os
import urllib

import numpy as np
import tensorflow as tf
import pandas as pd


IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "https://s3.amazonaws.com/elephantscale-public/data/iris/tensorflow/iris_training.csv"
IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "https://s3.amazonaws.com/elephantscale-public/data/iris/tensorflow/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

BATCH_SIZE=10




## The Iris classification problem

Imagine you are a botanist seeking an automated way to categorize each Iris flower you find. Machine learning provides many algorithms to statistically classify flowers. For instance, a sophisticated machine learning program could classify flowers based on photographs. Our ambitions are more modest—we're going to classify Iris flowers based on the length and width measurements of their [sepals](https://en.wikipedia.org/wiki/Sepal) and [petals](https://en.wikipedia.org/wiki/Petal).

The Iris genus entails about 300 species, but our program will only classify the following three:

* Iris setosa
* Iris virginica
* Iris versicolor

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/iris_three_species.jpg"
         alt="Petal geometry compared for three iris species: Iris setosa, Iris virginica, and Iris versicolor">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://commons.wikimedia.org/w/index.php?curid=170298">Iris setosa</a> (by <a href="https://commons.wikimedia.org/wiki/User:Radomil">Radomil</a>, CC BY-SA 3.0), <a href="https://commons.wikimedia.org/w/index.php?curid=248095">Iris versicolor</a>, (by <a href="https://commons.wikimedia.org/wiki/User:Dlanglois">Dlanglois</a>, CC BY-SA 3.0), and <a href="https://www.flickr.com/photos/33397993@N05/3352169862">Iris virginica</a> (by <a href="https://www.flickr.com/photos/33397993@N05">Frank Mayfield</a>, CC BY-SA 2.0).<br/>&nbsp;
  </td></tr>
</table>

Fortunately, someone has already created a [data set of 120 Iris flowers](https://en.wikipedia.org/wiki/Iris_flower_data_set) with the sepal and petal measurements. This is a classic dataset that is popular for beginner machine learning classification problems.

In [ ]:

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
   
    train = pd.read_csv(IRIS_TRAINING_URL, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(IRIS_TEST_URL, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y, test_x, test_y)

train_x, train_y, test_x, test_y = load_data()


In [ ]:
# Let's take a look at the data

pd.read_csv(IRIS_TRAINING_URL, names=CSV_COLUMN_NAMES, header=0)



## Build the Feature Columns

In [ ]:
# Specify that all features have real-value data
feature_columns = []
for key in train_x.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

## Build the Neural Network:

**=> TODO: build a 3 layer network with hidden layer as follows **

* First Layer: 10 neurons
* Second Layer: 20 Neurons
* Third Layer: 10 neurons

In [ ]:

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

In [ ]:
# Define the input function

def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


**=> TODO: Run for 1000 epochs**

In [ ]:

# Fit model.
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE), 
    steps=???)



In [ ]:

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    
    features=dict(features)
    inputs = features if labels is None else (features, labels)
   
    # Convert the inputs to a Dataset.
    return tf.data.Dataset.from_tensor_slices(inputs).batch(batch_size)


# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x, test_y[:3], batch_size=BATCH_SIZE))

list(predictions)

## Interpret The results

Look at the probability output above. There were 3 sample shown, with the probabilities. For two of them, our model had a great deal of certainty of its response. For the last, the certainty was still high but somewhat less.

Which model had the greatest deal of uncertainty? (Note that certainty is still quite high).  If not the predicted value, which is the likely alternative we could consider?